In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 

# Setting up the Dataset & Workflow

In [3]:
# Path to the folder where the datasets are/should be downloaded (e.g. MNIST)
DATASET_PATH = "./data"

# Function for setting the seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
set_seed(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [4]:
data = load_breast_cancer()

In [6]:
features = data.data
labels = data.target

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Transforms
## Using scikit this time 

In [7]:
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

In [8]:
features_train = torch.tensor(features_train, dtype=torch.float32).to(device)
labels_train = torch.tensor(labels_train, dtype=torch.float32).to(device)
features_test = torch.tensor(features_test, dtype=torch.float32).to(device)
labels_test = torch.tensor(labels_test, dtype=torch.float32).to(device)

# Making our Neural Network here

In [15]:
class BreastCancerNN(nn.Module):
    def __init__(self):
        super(BreastCancerNN, self).__init__()
        self.fc1 = nn.Linear(features_train.shape[1], 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [16]:
model = BreastCancerNN()

In [17]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
model.to(device)
def train_model(model, num_epochs=117):
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        model.train()
        output = model(features_train)
        loss = loss_fn(output, labels_train.view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            pred = output.round()
            correct = (pred == labels_train.view(-1,1)).float().sum()
            accuracy = correct/labels_train.size(0)

        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

In [21]:
train_model(model)

  0%|          | 0/117 [00:00<?, ?it/s]

Epoch [10/117], Loss : 0.5615, Accuracy: 90.11%
Epoch [20/117], Loss : 0.4435, Accuracy: 94.51%
Epoch [30/117], Loss : 0.3507, Accuracy: 94.51%
Epoch [40/117], Loss : 0.2791, Accuracy: 94.95%
Epoch [50/117], Loss : 0.2268, Accuracy: 95.38%
Epoch [60/117], Loss : 0.1900, Accuracy: 96.04%
Epoch [70/117], Loss : 0.1638, Accuracy: 96.70%
Epoch [80/117], Loss : 0.1445, Accuracy: 96.48%
Epoch [90/117], Loss : 0.1297, Accuracy: 96.70%
Epoch [100/117], Loss : 0.1181, Accuracy: 96.92%
Epoch [110/117], Loss : 0.1087, Accuracy: 97.36%


In [22]:
model_name=type(model).__name__
saved_model = torch.save(model.state_dict(), model_name+'_model.pth')

In [25]:
def test_model(model):
    model.eval()
    with torch.no_grad():
        output = model(features_test)
        pred = output.round()
        correct = (pred == labels_test.view(-1,1)).float().sum()
        accuracy = correct/labels_test.size(0)
        print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

In [26]:
test_model(model)

Accuracy on test data: 98.25%
